Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру. Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [1]:
with open('checkins.dat') as input_file:        
    newLines = []
    for line in input_file:
        newLine = [x.strip() for x in line.split('|')]
        if len(newLine) == 6 and newLine[3] and newLine[4]:
            newLines.append(newLine)

In [2]:
import csv

with open('checkins.csv', 'w') as checkins_csv:
    writer = csv.writer(checkins_csv)
    writer.writerows(newLines)

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [3]:
import pandas as pd

In [4]:
checkins_data = pd.read_csv('checkins.csv')
checkins_data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

**Примечание**: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [5]:
from sklearn.cluster import MeanShift

In [6]:
X = checkins_data.values
limit = 100000
X = X[:limit]
print X.shape
X_coords = X[:, 3:5]
print X_coords.shape
print X_coords[:2]

(100000, 6)
(100000, 2)
[[38.895111799999995 -77.0363658]
 [33.800745 -84.41051999999999]]


In [7]:
ms = MeanShift(bandwidth=0.1, bin_seeding=True)
ms.fit(X_coords)

MeanShift(bandwidth=0.1, bin_seeding=True, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [9]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_
print labels
print cluster_centers

[ 5  7 28 ..., 23 19  4]
[[  40.7177164   -73.99183542]
 [  33.44638027 -111.90188756]
 [  33.44841049 -112.07400428]
 ..., 
 [  36.481464    -94.2732642 ]
 [  29.6899563   -95.8996757 ]
 [  31.3787916   -95.3213317 ]]


In [19]:
print labels.shape
print cluster_centers.shape

(100000,)
(3091, 2)


In [20]:
print labels[:10]

[ 5  7 28 64  2 22  0  2  1  8]


In [13]:
import numpy as np
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
print n_clusters_

3091


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [15]:
labels_unique[:5]

array([0, 1, 2, 3, 4])

In [16]:
labels_unique, counts = np.unique(labels, return_counts=True)

In [18]:
counts[:3]

array([12511,  4009,  4677])

In [25]:
centers_count = zip(labels_unique, counts)
print centers_count[:3]

[(0, 12511), (1, 4009), (2, 4677)]


In [27]:
centers_count_more15 = [center for center in centers_count if center[1] > 15]

In [28]:
len(centers_count_more15)

589

In [29]:
print centers_count_more15[:20]

[(0, 12511), (1, 4009), (2, 4677), (3, 3363), (4, 3526), (5, 2445), (6, 2297), (7, 1601), (8, 1532), (9, 1378), (10, 1298), (11, 1164), (12, 1012), (13, 1008), (14, 868), (15, 714), (16, 870), (17, 645), (18, 808), (19, 807)]


In [33]:
centers_count_more15.sort(key = lambda center: center[1], reverse = True)
print centers_count_more15[:20]

[(0, 12511), (2, 4677), (1, 4009), (4, 3526), (3, 3363), (5, 2445), (6, 2297), (7, 1601), (8, 1532), (9, 1378), (10, 1298), (11, 1164), (12, 1012), (13, 1008), (16, 870), (14, 868), (18, 808), (19, 807), (21, 754), (22, 747)]


In [35]:
offices_coords = np.array([[33.751277, -118.188740], 
                     [25.867736, -80.324116], 
                     [51.503016, -0.075479], 
                     [52.378894, 4.885084], 
                     [39.366487, 117.036146], 
                     [-33.868457, 151.205134]])


In [59]:
dist_list = []
for center in centers_count_more15:
    for office_coords in offices_coords:
        center_coords = cluster_centers[center[0]]
        dist = np.linalg.norm(office_coords-center_coords)
        dist_list.append((dist, center[0]))

In [60]:
print dist_list[:10]
dist_list.sort()
print dist_list[:10]

[(44.742570917848042, 0), (16.14371999090071, 0), (74.699065816874452, 0), (79.734255375310127, 0), (191.03276029583111, 0), (237.22725875825117, 0), (6.1222317053408144, 2), (32.642334958098395, 2), (113.44442888391637, 2), (118.48118642629552, 2)]
[(0.007834758163107856, 411), (0.0093533161859922255, 373), (0.022358439051973395, 302), (0.050058294822787869, 58), (0.070847732427199731, 51), (0.13410903336184654, 25), (0.16740596425035326, 168), (0.18653597623002971, 192), (0.18887596060185083, 91), (0.19577945647763628, 85)]


In [61]:
print cluster_centers[411]

[ -33.86063043  151.20477593]


In [67]:
def write_answer(number, answer):
    with open("answer{}.txt".format(number), "w") as fout:
        fout.write(str(answer))


write_answer(1, cluster_centers[411])
